In [2]:
import tensorflow as tf
import numpy as np

In [20]:
nbatch = 2
nunroll = 3
nsyms = 4
logits_ = np.random.rand(nbatch, nunroll, nsyms).astype(np.float32)
logits_exp_ = np.exp(logits_)
softmax_ = np.divide(logits_exp_, np.sum(logits_exp_, axis=2)[:, :, np.newaxis])
weights_ = np.ones((nbatch, nunroll), dtype=np.float32)
weights_[0][2:] = 0.0
targets_ = np.random.randint(1, nsyms, size=(nbatch, nunroll), dtype=np.int32)
targets_[0][2:] = 0
print logits_
print targets_
print weights_
print softmax_

[[[ 0.47135267  0.29029804  0.89198583  0.46804243]
  [ 0.6934647   0.23503804  0.70438915  0.7256664 ]
  [ 0.00224746  0.77212453  0.22418016  0.89110726]]

 [[ 0.25514391  0.47887799  0.01040162  0.88948047]
  [ 0.89030439  0.81758666  0.41806781  0.71011662]
  [ 0.23843014  0.83375996  0.16533574  0.42635858]]]
[[1 2 0]
 [3 1 3]]
[[ 1.  1.  0.]
 [ 1.  1.  1.]]
[[[ 0.2296733   0.19163707  0.34977534  0.22891431]
  [ 0.27203569  0.17200209  0.27502385  0.2809383 ]
  [ 0.14619151  0.31570065  0.18251824  0.35558963]]

 [[ 0.2032762   0.25424534  0.15914619  0.38333225]
  [ 0.29510862  0.27441069  0.1840315   0.24644917]
  [ 0.20202473  0.36639792  0.18778463  0.24379274]]]


In [33]:
seq1_neg_log_prob = -1.0 * np.log([0.19163707, 0.27502385])
seq2_neg_log_prob = -1.0 * np.log([0.38333225, 0.27441069, 0.24379274])
print seq1_neg_log_prob
print seq2_neg_log_prob
seq1_sum_log_prob = np.sum(seq1_neg_log_prob)
seq2_sum_log_prob = np.sum(seq2_neg_log_prob)
print seq1_sum_log_prob
print seq2_sum_log_prob
seq1_avg_neg_log_prob = (1.0 / 2) * seq1_sum_log_prob
seq2_avg_neg_log_prob = (1.0 / 3) * seq2_sum_log_prob
print seq1_avg_neg_log_prob
print seq2_avg_neg_log_prob
loss = np.mean([seq1_avg_neg_log_prob, seq2_avg_neg_log_prob])
print loss

[ 1.65215196  1.29089746]
[ 0.95885317  1.29312943  1.41143684]
2.94304941397
3.66341943943
1.47152470699
1.22113981314
1.34633226006


In [31]:
splitsqueeze = lambda x: [tf.squeeze(r, [1]) for r in tf.split(1, nunroll, x)]
splitsqueezeflat = lambda x: [tf.reshape]

with tf.Graph().as_default(), tf.Session() as sess:
    logits = tf.constant(logits_)
    targets = tf.constant(targets_)
    weights = tf.constant(weights_)
    
    posterior = tf.nn.softmax(tf.reshape(logits, [nbatch * nunroll, nsyms]))
    posterior = tf.reshape(posterior, [nbatch, nunroll, nsyms])

    neg_log_probs = tf.nn.seq2seq.sequence_loss_by_example(
        splitsqueeze(logits),
        splitsqueeze(targets),
        splitsqueeze(weights))
    
    neg_log_probs_flat = tf.nn.seq2seq.sequence_loss_by_example(
        [tf.reshape(logits, [nbatch * nunroll, nsyms])],
        [tf.reshape(targets, [-1])],
        [tf.reshape(weights, [-1])])
    
    print posterior.eval()
    neg_log_probs = neg_log_probs.eval()
    neg_log_probs_flat = neg_log_probs_flat.eval()
    print neg_log_probs
    print neg_log_probs_flat

[[[ 0.22967331  0.19163705  0.34977534  0.22891429]
  [ 0.27203575  0.17200212  0.27502388  0.28093833]
  [ 0.14619149  0.31570065  0.18251824  0.3555896 ]]

 [[ 0.20327619  0.25424534  0.1591462   0.38333228]
  [ 0.29510862  0.27441067  0.18403149  0.24644916]
  [ 0.20202473  0.36639789  0.18778461  0.24379276]]]
[ 1.47152472  1.22113979]
[ 1.65215194  1.29089737  0.          0.95885313  1.29312944  1.4114368 ]


In [35]:
print np.sum(neg_log_probs) / neg_log_probs.shape[0]

1.34633231163
